# Drive Train Parameters

As we design the alt-az mount, there are a bunch of options for designs, and the decision comes down to what kind of drive train we want to use.  That decision is in turn driven by the tracking precision needed in our system.  That, in turn, is decided by the optical parameters for exposure.

First up, our field of view: this is a function of the F.O.V. (aka Angle Of View) equation: $\frac{w}{f}\cdot\frac{180}{\pi}$, with $w$ the width of the sensor, $f$ the focal length of the scope, and an angle small enough that the small angle approximation holds (otherwise there's a bit of trig in there).  In our case, $f = 1000\textrm{mm}$, so this turns into simply $w_m \textrm{rad}$.  For the Canon EOS APS-C sensor, $w = (22.3,14.9)\textrm{mm}$, which leads to a FOV of (1.28°×0.85°).

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
f_mm = 1000
w_mm = np.array([22.3, 14.9])

fov_deg = w_mm/f_mm * 180.0/np.pi
fov_deg

array([ 1.27769588,  0.85370711])

The next thing to consider is the pixels at work: our sensor is 4,752×3,168, which leads to a pixel pitch of 4.69µm in both directions.  We can either divide the FOV from the previous step by the pixels, or just use the pixel pitch.  Either way, we get an angular resolution of 0.97 arcseconds per pixel on our sensor.

In [4]:
w_px = np.array([4752,3168])
angres1_px = fov_deg/w_px * 3600

angres_px = 4.69e-6*180/np.pi * 3600

angres1_px, angres_px

(array([ 0.96795143,  0.97012172]), 0.9673819412988819)

However, we'd be thoroughly blessed to get seeing that's better than 3", so 0.97"/pixel means that our average feature is roughly a 3×3 pixel blotch on the sensor, which we'll factor in later.  For now, let's find out how far the objects are going to be moving, in pixels, during each exposure.

We need to pull the average azimuthal speeds from `passes_to_parameters.ipynb` in this directory.  There, we get azimuthal rates with a median of 0.03573°/s and a mean of 0.152081°/s.  Applying Tukey inner fences, we get a mean of 0.05°/s, so the 0.15°/s is a pretty aggressive pass.  However, looking at ISS data, it seems that our mean is closer to 0.2°/s, so let's shoot for that.

There is a wide range of exposure options here, so we'll look at that as well.

In [6]:
from IPython.display import display,HTML

In [28]:
class Optics:
    def __init__(self, nom, w_px,h_px, w_mm,h_mm, fps, f_mm):
        self.nom = nom
        self.px = np.array([w_px, h_px])
        self.mm = np.array([w_mm, h_mm])
        self.fps = fps
        self.f_mm = f_mm
        
    def fov(self):
        k = 180.0/np.pi * 1.0/self.f_mm
        return k * self.mm
    
    def fov_px(self):
        return self.fov()/self.px
    
    def px_per_frame(self, speed_degps):
        return speed_degps/fps / self.fov_px()        

In [29]:
f_mm = 1000.0 # telescope holds this kinda constant...

# Webcam looks to be a 1/3" display: 4.8x3.6mm
wcw_px = 4.8
wch_px = 3.6

f_mm = 1000.0

optics = [
    Optics("Canon 500D 1080p", 1920,1080, 22.3,14.9, 20.0, f_mm),
    Optics("Canon 500D 720p",  1280, 720, 22.3,14.9, 30.0, f_mm),
    Optics("Canon 500D 1/3",   4752,3168, 22.3,14.9,  3.0, f_mm),
    Optics("Webcam VGA",        640, 480, wcw_px,wch_px, 30.0, f_mm),
    Optics("Webcam SVGA",       800, 600, wcw_px,wch_px, 20.0, f_mm),
    Optics("Webcam UVGA",       1280, 960, wcw_px,wch_px,  9.0, f_mm),
    Optics("Webcam UXGA",      1600,1200, wcw_px,wch_px,  5.0, f_mm),
]


In [31]:
speed_degps = 0.2

# This gets more complicated with resolutions, though, so we'll factor that in


table = "<table><tr>" + "".join(["<th>%s</th>" % s for s in "Camera FPS FOVs/sec Pixels/Frame FOV/px".split() ]) + "</tr>"

for o in optics:
    row = o.nom, o.fps, speed_degps/o.fov()[0], o.px_per_frame(speed_degps)[0], o.fov_px()[0]
    table += "<tr><td>" + "</td><td>".join(map(str, row)) + "</td></tr>"
    
table += "</table>"
display(HTML(table))

Camera,FPS,FOVs/sec,Pixels/Frame,FOV/px
Canon 500D 1080p,20.0,0.156531771479,60.1082002481,0.000665466605803
Canon 500D 720p,30.0,0.156531771479,40.0721334987,0.000998199908704
Canon 500D 1/3,3.0,0.156531771479,148.767795614,0.000268875396284
Webcam VGA,30.0,0.727220521664,93.084226773,0.000429718346348
Webcam SVGA,20.0,0.727220521664,116.355283466,0.000343774677078
Webcam UVGA,9.0,0.727220521664,186.168453546,0.000214859173174
Webcam UXGA,5.0,0.727220521664,232.710566933,0.000171887338539


I'm starting to think that this is all inside out, and I should be starting with the precision of the TLEs and sorting out the range of field I need to be able to see in order to best assess their validity… the above is all about the direct imaging case, but that's possibly beyond my current equipment anyway.

## Closed loop sensor precision

Another question that occurs to me: how precise do my sensors need to be to track things accurately?  Let's start with the most demanding case: pixel-perfect tracking.

There are actually two questions here, one for altitude and the other for azimuth.  We'll assume the camera sensor is arranged in "landscape" mode, and focus on the azimuth case, as its markedly harder than the altitude case.

(Why is it so much harder?  While the sensor is 33% bigger, it needs to cover 360° of azimuth, while the altitude only has to go from 0° to 90°.)

In [32]:
table = "<table><tr><th>" + "</th><th>".join(["Camera", "arcsec/px", "Total steps"]) + "</th></tr>"

for o in optics:
    row = [ o.nom, o.fov_px()[0]*3600, int(360.0/o.fov_px()[0]) ]
    table += "<tr>" + "".join(["<td>%s</td>" % s for s in map(str, row) ]) + "</tr>"

table += "</table>"
display(HTML(table))

Camera,arcsec/px,Total steps
Canon 500D 1080p,2.39567978089,540973
Canon 500D 720p,3.59351967134,360649
Canon 500D 1/3,0.967951426623,1338910
Webcam VGA,1.54698604685,837758
Webcam SVGA,1.23758883748,1047197
Webcam UVGA,0.773493023427,1675516
Webcam UXGA,0.618794418741,2094395


Yeah, so... that seems unlikely to happen.  Two million steps per revolution is rather a lot, no matter what the control system is.  Random question: what kind of gearing ratio is that for a stepper?

In [35]:
target_steps = 360.0/(optics[-1].fov_px()[0])

steps_per_rev = 200 # typical 1.8deg steps

target_steps/steps_per_rev

10471.975511965977

10,000:1 is kind of a lot.  Though it's possible to microstep 10:1 (GeckoDrives), which leaves us with 1000:1, which is a couple of 30:1 reductions cascaded.  It's not impossible to get that precision with steppers.

What about the other end of the precision scale: getting to within a third of the frame (going to "within half a frame" means that the object may be on the edge of the frame instead of in the center, so let's go with 3x) ?

In [38]:
table = "<table><tr><th>" + "</th><th>".join(["Camera", "arcsec/frame", "Total steps"]) + "</th></tr>"

for o in optics:
    r = o.fov()[0]/3
    row = [ o.nom, r*3600, int(360.0/r) ]
    table += "<tr>" + "".join(["<td>%s</td>" % s for s in map(str, row) ]) + "</tr>"

table += "</table>"
display(HTML(table))

Camera,arcsec/frame,Total steps
Canon 500D 1080p,1533.23505977,845
Canon 500D 720p,1533.23505977,845
Canon 500D 1/3,1533.23505977,845
Webcam VGA,330.023689995,3926
Webcam SVGA,330.023689995,3926
Webcam UVGA,330.023689995,3926
Webcam UXGA,330.023689995,3926


Interestingly, this is only a bit better: we still need gearing to get this resolution with steppers.  Admittedly, it's a much more manageable 20:1 or so.  Now for the fun question: how many pixels wide is a given step at various gearing ratios?

In [55]:
ratios = [1, 3, 5, 10, 20, 30, 50, 100, 200, 500, 1000, 3000, 10000 ]

table = "<table><caption>Pixels per step, at various gear ratios</caption>"
table += "<tr><th>" + "</th><th>".join(["Camera"] + map(str, ratios)) + "</th></tr>"

for o in optics:    
    r = o.fov_px()[0]
    row = [ o.nom ] + [ "%0.1f" % (360.0/(200.0*ratio) / r) for ratio in ratios ]
    table += "<tr>" + "".join(["<td>%s</td>" % s for s in map(str, row) ]) + "</tr>"

table += "</table>"
display(HTML(table))

Camera,1,3,5,10,20,30,50,100,200,500,1000,3000,10000
Canon 500D 1080p,2704.9,901.6,541.0,270.5,135.2,90.2,54.1,27.0,13.5,5.4,2.7,0.9,0.3
Canon 500D 720p,1803.2,601.1,360.6,180.3,90.2,60.1,36.1,18.0,9.0,3.6,1.8,0.6,0.2
Canon 500D 1/3,6694.6,2231.5,1338.9,669.5,334.7,223.2,133.9,66.9,33.5,13.4,6.7,2.2,0.7
Webcam VGA,4188.8,1396.3,837.8,418.9,209.4,139.6,83.8,41.9,20.9,8.4,4.2,1.4,0.4
Webcam SVGA,5236.0,1745.3,1047.2,523.6,261.8,174.5,104.7,52.4,26.2,10.5,5.2,1.7,0.5
Webcam UVGA,8377.6,2792.5,1675.5,837.8,418.9,279.3,167.6,83.8,41.9,16.8,8.4,2.8,0.8
Webcam UXGA,10472.0,3490.7,2094.4,1047.2,523.6,349.1,209.4,104.7,52.4,20.9,10.5,3.5,1.0


In [56]:
table = "<table><caption>Fraction of sensor per step, at various gear ratios</caption>"
table += "<tr><th>" + "</th><th>".join(["Camera"] + map(str, ratios)) + "</th></tr>"

for o in optics:    
    r = o.fov()[0]
    row = [ o.nom ] + [ "%0.4f" % (360.0/(200.0*ratio) / r) for ratio in ratios ]
    table += "<tr>" + "".join(["<td>%s</td>" % s for s in map(str, row) ]) + "</tr>"

table += "</table>"
display(HTML(table))

Camera,1,3,5,10,20,30,50,100,200,500,1000,3000,10000
Canon 500D 1080p,1.4088,0.4696,0.2818,0.1409,0.0704,0.0470,0.0282,0.0141,0.0070,0.0028,0.0014,0.0005,0.0001
Canon 500D 720p,1.4088,0.4696,0.2818,0.1409,0.0704,0.0470,0.0282,0.0141,0.0070,0.0028,0.0014,0.0005,0.0001
Canon 500D 1/3,1.4088,0.4696,0.2818,0.1409,0.0704,0.0470,0.0282,0.0141,0.0070,0.0028,0.0014,0.0005,0.0001
Webcam VGA,6.5450,2.1817,1.3090,0.6545,0.3272,0.2182,0.1309,0.0654,0.0327,0.0131,0.0065,0.0022,0.0007
Webcam SVGA,6.5450,2.1817,1.3090,0.6545,0.3272,0.2182,0.1309,0.0654,0.0327,0.0131,0.0065,0.0022,0.0007
Webcam UVGA,6.5450,2.1817,1.3090,0.6545,0.3272,0.2182,0.1309,0.0654,0.0327,0.0131,0.0065,0.0022,0.0007
Webcam UXGA,6.5450,2.1817,1.3090,0.6545,0.3272,0.2182,0.1309,0.0654,0.0327,0.0131,0.0065,0.0022,0.0007


In [57]:
table = "<table><caption>Arcseconds per step, at various gear ratios</caption>"
table += "<tr><th>" + "</th><th>".join(map(str, ratios)) + "</th></tr>"

row = [ "%0.2f" % (360.0*3600/(200.0*ratio)) for ratio in ratios ]
table += "<tr>" + "".join(["<td>%s</td>" % s for s in map(str, row) ]) + "</tr>"

table += "</table>"
display(HTML(table))

1,3,5,10,20,30,50,100,200,500,1000,3000,10000
6480.00,2160.00,1296.00,648.00,324.00,216.00,129.60,64.80,32.40,12.96,6.48,2.16,0.65


I think a key takeaway here is that it takes 1,000:1 gearing just to get into the ballpark of good seeing.  That's going to be a many step geartrain (unless I get really goofy), which means that it will have its own backlash/slop.  This is going to vary with the angle the scope is at etc, which makes for a bit of a control mess.

There are planetary gearboxes available on NEMA steppers at reasonable prices, all the way up to 100:1 ratios, which means that we might be able to get to something like 3,000:1 with a subsequent stage, with relatively tight amounts of slop.

Whatever the outcome: this level of control looks particularly hairy, and means we definitely have to work with steps that are much much larger than our targets. This precludes the idea of tracking an object within a set of pixels, and forces a much more slew, capture, slew, capture sort of affair.